In [1]:
from Database.Database import DataFunFilm , DataFunDist
from gensim.similarities import WmdSimilarity
from Handlers.Descriptions import Model
from datetime import time
from tqdm import tqdm
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

In [2]:
distDB = DataFunDist()
db = DataFunFilm()
films = db.getTableDF()
model = Model()

In [3]:
films

,id,kinopoisk_id,imdb_id,name_ru,name_original,poster_url,rating_imdb,rating_imdb_vote_count,rating_film_critics,rating_critics_vote_count,year,film_length,genres,web_url,slogan,description,rating_kinopoisk,short_desription,rating_age_limits,prepare_description
0,1,1043658,tt4202274,Шоу Грязного Фрэнка,The Filthy Frank Show,https://kinopoiskapiunofficial.tech/images/pos...,9.7,32929,NaN,0,2011.0,12.0,фэнтези комедия музыка короткометражка,https://www.kinopoisk.ru/series/1043658/,None,Шоу повествует о героической борьбе сэра Фрэнс...,9.3,None,None,шоу_PROPN повествовать_VERB о_ADP героический...
1,2,1309325,None,Космос,,https://kinopoiskapiunofficial.tech/images/pos...,NaN,0,NaN,0,2019.0,NaN,документальный,https://www.kinopoisk.ru/series/1309325/,None,Астрофизик Нил Деграсс Тайсон встает за штурва...,9.3,Нил Деграсс Тайсон отправляется в мысленное пу...,age18,астрофизик_NOUN нил_VERB деграсс::тайсон_PROPN...
2,3,77044,tt0108778,Друзья,Friends,https://kinopoiskapiunofficial.tech/images/pos...,9.0,937875,NaN,0,1994.0,22.0,мелодрама комедия,https://www.kinopoisk.ru/series/77044/,Everybody needs Friends!,"Шестеро друзей – Рейчел, Моника, Фиби, Джоуи, ...",9.2,"Шестеро друзей, две квартиры и много юмора. Си...",age16,шестеро_NUM друг_NOUN рейчел_PROPN моника_PRO...
3,4,972889,tt4926348,BadComedian,None,https://kinopoiskapiunofficial.tech/images/pos...,7.7,793,NaN,0,2011.0,NaN,комедия,https://www.kinopoisk.ru/series/972889/,None,"Юмористическое шоу, высмеивающее плохое кино. ...",9.2,None,age18,юмористический_ADJ шоу_NOUN высмеивать_VERB пл...
4,5,674243,None,Счастливые люди,None,https://kinopoiskapiunofficial.tech/images/pos...,8.8,120,NaN,0,2008.0,55.0,документальный,https://www.kinopoisk.ru/series/674243/,None,Документальный фильм о трудах и днях охотников...,9.2,None,None,документальный_ADJ фильм_NOUN о_ADP труд_NOUN ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66752,66753,83307,tt0041640,Мари из порта,La Marie du port,https://kinopoiskapiunofficial.tech/images/pos...,6.7,395,NaN,0,1950.0,88.0,драма мелодрама,https://www.kinopoisk.ru/film/83307/,None,"Анри Шателяру уже за сорок, он владеет рестора...",7.0,None,None,анри_PROPN шателяр_PROPN уже_ADV за_ADP сорок...
66753,66754,219369,tt0071859,Путешественник,Mossafer,https://kinopoiskapiunofficial.tech/images/pos...,7.5,1695,NaN,0,1974.0,83.0,драма,https://www.kinopoisk.ru/film/219369/,None,Парнишке-школьнику по имени Кассем из маленько...,7.0,None,None,парнишка_NOUN школьник_NOUN по_ADP имя_NOUN ка...
66754,66755,47614,tt0211372,Сын француза,Le fils du Français,https://kinopoiskapiunofficial.tech/images/pos...,5.0,282,NaN,0,1999.0,107.0,приключения комедия семейный,https://www.kinopoisk.ru/film/47614/,None,У Бенжамена 9 лет нет больше мамы. Он хочет на...,7.0,None,None,в_ADP бенжамен_PROPN 9_NUM год_NOUN нет_VERB ...
66755,66756,977755,tt9378362,Звоните ДиКаприо!,None,https://kinopoiskapiunofficial.tech/images/pos...,6.7,496,NaN,0,2018.0,48.0,драма комедия,https://www.kinopoisk.ru/series/977755/,Что останется после тебя?,Егор Румянцев – звезда популярного сериала про...,7.0,None,age18,егор::румянцев_PROPN звезда_NOUN популярный_A...


In [4]:
model_descr = list(films["prepare_description"])

In [5]:
description = list(map(lambda x : x.split(),list(films["prepare_description"])))
description[0]

['шоу_PROPN',
 'повествовать_VERB',
 'о_ADP',
 'героический_ADJ',
 'борьба_NOUN',
 'сэр_NOUN',
 'фрэнсис_PROPN',
 'грязной_ADJ',
 'и_CCONJ',
 'он_PRON',
 'верный_ADJ',
 'друг_NOUN',
 'против_ADP',
 'зловещий_ADJ',
 'темный_ADJ',
 'бог_NOUN',
 'в_ADP',
 'сюрреалистический_ADJ',
 'омнивселенной_NOUN']

In [6]:
keys = model.model.key_to_index.keys()

In [64]:
desc = []
for i in tqdm(range(len(description))):
    sent = []
    for word in description[i]:
        if word in keys:
            sent.append(model.model[word])
    desc.append(np.array(sent))
desc = np.array(desc, dtype="object")
desc[0]

100%|██████████| 66757/66757 [00:16<00:00, 4030.19it/s]


array([[ 0.6778329 , -2.1406288 ,  0.23269792, ..., -1.0962808 ,
        -1.3404318 ,  0.877907  ],
       [-0.94731104, -0.99548364, -1.3306068 , ..., -2.2272267 ,
         0.94829875,  0.18321598],
       [-1.4011941 , -0.9699115 ,  0.3996764 , ..., -0.5424539 ,
        -0.00978736, -1.1246781 ],
       ...,
       [-2.0932076 , -0.3544742 ,  0.946783  , ...,  0.26891842,
        -3.046455  , -1.3474947 ],
       [-0.98503155, -1.751591  ,  0.51085794, ..., -2.618336  ,
        -2.2761512 , -3.376037  ],
       [ 0.12781557, -0.1343181 , -0.0792335 , ..., -0.783275  ,
         1.2887568 , -1.6977879 ]], dtype=float32)

In [9]:
model = SentenceTransformer('all-mpnet-base-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [11]:
%%time
sentence_embeddings = model.encode(model_descr)
sentence_embeddings.shape


KeyboardInterrupt



In [66]:
index = faiss.IndexHNSW(model.model.vector_size)
index.add(desc)

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
index.add(model.model.vectors)

In [50]:
index.ntotal

249333

In [51]:
index.is_trained

True

In [52]:
k = 10
xq = desc[0]

In [53]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 13430  60244  88831 176302]
 [  7657  19040    921  13982]
 [  6586  84816  30647  12363]
 [   429   4904  13797  29807]
 [  3734   5634   4603   7066]
 [ 16317  21654  63775  14863]
 [  1882  20924  16784   3831]
 [   146  15344  77577  42360]
 [ 12816  57420   8164  60713]
 [  1457   2984   1775   1727]
 [   881   5384   5636    994]
 [ 30176 131517  53329 109166]]
CPU times: total: 1.5 s
Wall time: 150 ms


In [47]:
films.iloc[1828]

id                                                                        1829
kinopoisk_id                                                            134703
imdb_id                                                              tt0347718
name_ru                                                            Другие миры
name_original                                                  D'autres mondes
poster_url                   https://kinopoiskapiunofficial.tech/images/pos...
rating_imdb                                                                7.6
rating_imdb_vote_count                                                     354
rating_film_critics                                                        NaN
rating_critics_vote_count                                                    0
year                                                                    2004.0
film_length                                                               90.0
genres                                              